In [ ]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import Dataset

In [ ]:
texts = ["I am so excited about this achievement!", 
         "Humbled to announce my new job", 
         "Normal day, normal post",
         "This is just a regular update.",
         "Thrilled to share I got a promotion!"]

labels = [1, 1, 0, 0, 1]  # 1=brag, 0=not brag

data = Dataset.from_dict({"text": texts, "label": labels})

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

data = data.map(tokenize, batched=True)
data.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
split = data.train_test_split(test_size=0.2)
train_ds = split["train"]
test_ds = split["test"]

# 3. Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_steps=10,
    evaluation_strategy="epoch"
)

# 5. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds
)

# Train
trainer.train()

In [ ]:
dummy_input = tokenizer("Hello world!", return_tensors="pt")
torch.onnx.export(
    model,
    (dummy_input["input_ids"], dummy_input["attention_mask"]),
    "model.onnx",
    input_names=["input", "mask"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "mask": {0: "batch_size"}, "output": {0: "batch_size"}},
    opset_version=11
)

print("Exported model to ONNX: model.onnx")